# TP exploration données stations d'observations

Le but de ce TP est d'apprendre à explorer les données d'observations des stations sol. Elles sont équipées de différents capteurs :
* température
* humidité 
* vent
* ...etc

Vous allez apprendre à calculer différentes statistiques (min, max, moyennes...) et à tracer vos résultats sous forme de courbes en Python.

## Chargement des librairies nécessaires


In [ ]:
import pandas as pd                 #librairie pour la manipulation de données
import numpy as np                  #librairie pour le calcul
import matplotlib.pyplot as plt     #librairie pour le tracé 

#librairies pour la gestion des dates
from datetime import datetime  
from datetime import timedelta  

## Aperçu des données

Chargement en mémoire des données (zone nord-ouest de la France, année 2016)

In [ ]:
fname = '/kaggle/input/meteonet/NW_Ground_Stations/NW_Ground_Stations/NW_Ground_Stations_2016.csv'    #nom de notre fichier csv 
df = pd.read_csv(fname,parse_dates=[4],infer_datetime_format=True)                                  #lecture du fichier

Les données sont stockées sous la forme d'un dataframe qui a une structure de tableau. 
Vous avez une observation par ligne et divers paramètres par colonne :

* **number_sta** : numéro de la station sol
* **lat** : latitude de la station en degrés nord
* **lon** : longitude de la station en degrès est
* **height_sta** : hauteur de la station en mètres
* **date** : date de l'observation
* **dd** : direction du vent en degrès
* **ff** : force du vent en m/s
* **precip**  : quantité de précipitations en mm
* **hu** : humidité en %
* **td** : température du point de rosée (en Kelvins) -> température de condensation 
* **t** : température en Kelvins
* **psl** : pression au niveau de la mer (en Pascal)

In [ ]:
#affichage de notre tableau de données 
display(df.head())     #1eres lignes 
display(df.tail())     #dernières lignes

Tout d'abord, choisissons une date puis récupérons uniquement les observations à cet instant:

In [ ]:
date = '2016-01-01T06:00:00'     #date choisie 
d_sub = df[df['date'] == date]   #filtrage sur la date choisie

#affichage des 1eres et dernières lignes 
display(d_sub.head())
display(d_sub.tail())

Choisissons un paramètre météo, puis traçons les données sur une carte 

In [ ]:
param = 'hu'                      #paramètre météo choisi (d'après les noms de colonnes)

#charger les librairies utiles pour le tracé du fond de carte (ici, la France)
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Coordonnées de la zone d'étude, quart nord-ouest de la France (in °N and °E)
lllat = 46.25  #lower left latitude
urlat = 51.896  #upper right latitude
lllon = -5.842  #lower left longitude
urlon = 2  #upper right longitude
extent = [lllon, urlon, lllat, urlat]

#Création de la fenêtre graphique
fig = plt.figure(figsize=(9,5))

# Sélectionner le type de planisphère souhaité : planisphère plat 
ax = plt.axes(projection=ccrs.PlateCarree())

# Tracer les données 
plt.scatter(d_sub['lon'], d_sub['lat'], c=d_sub[param], cmap='jet') 
plt.colorbar().set_label(param)   #échelle de couleur 
plt.title('date '+ date)         #titre

# Ajouter le fond de carte (traits de côtes...)
ax.coastlines(resolution='50m', linewidth=1)
ax.add_feature(cfeature.BORDERS.with_scale('50m'))

plt.show()

Pour une date donnée, cette carte représente les observations du paramètre météo choisi (si vous avez pris celui de l'exemple : l'humidité) sur le quart nord-ouest de la France. Chaque point symbolise une station d'observation et la couleur indique la valeur du paramètre. 

## Calcul de valeurs statistiques et tracés de courbes

### Récupération des données d'intérêt

Choisissons tout d'abord une station et un paramètre météo. 

In [ ]:
station_id = 14066001   #identifiant de la station choisie
param = 't'             #paramètre météo choisi

d_sub2 = df[(df['number_sta'] == station_id)][{'number_sta','date',param}]   #filtrage sur la station et le paramètre choisis
d_sub2

Ensuite, nous devons récupérer uniquement les données qui nous intéressent et les mettre sous la bonne forme.

In [ ]:
d_f = d_sub2.drop('number_sta', axis = 1)   #supprimer la colonne number_sta qui est maintenant inutile
#placer la colonne date en 'indice' -> considérée comme la colonne de référence. 
#Cela permet de faciliter les calculs car nous allons nous baser sur le temps (min, max et moyennes par mois)
d_f.set_index('date',inplace = True)    
d_f

Traçons maintenant les données sur toute l'année 2016

In [ ]:
plt.figure(figsize=(11,7))  #Création de la fenêtre graphique
plt.plot(d_f)   #Tracé de la courbe
plt.title('Station ' + str(station_id))   #titre 
plt.xlabel('Date')                        #légende de l'axe des abscisses 
plt.ylabel(param)                         #légénde de l'axe des ordonnées

Voici le tracé des données du paramètre choisi en fonction du temps. Si vous avez choisi la température, nous constatons bien que les températures sont plus élevées durant l'été que le reste de l'année. 

### Calcul des valeurs moyennes par mois

Calculons maintenant les valeurs moyennes par mois.

In [ ]:
moy = d_f.resample('M').mean()  # 'M' pour month qui signifie par mois et mean, moyenne
moy

Voilà, nous obtenons bien les valeurs moyennes par mois ! Cependant, petit hic : par défaut, la date pour chaque mois est celle du dernier jour du mois. Cela peut poser des problèmes de décalage au moment de tracer la courbe. Comme solution, nous pouvons se mettre au milieu du mois à la place. 

In [ ]:
moy2 = moy.reset_index()  #remettre la date comme 'colonne classique' (la retirer de l'index)
moy2['date'] = moy2['date'].apply(lambda x: x - timedelta(days=15))   #se mettre au milieu du mois : retirer 15 jours
moy2.set_index('date', inplace=True)      #remettre la date en index -> colonne de référence
moy2

Nous obtenons bien ce que nous souhaitons : une date de mileu de mois à chaque fois, ce qui va nous permettre de centrer les données sur la courbe pour chaque mois.

Pour pouvoir comparer l'ensemble des valeurs aux valeurs moyennes, traçons les 2 courbes sur un seul graphique

In [ ]:
plt.figure(figsize=(11,7))                  #taille de la fenêtre graphique
plt.plot(d_f, label = "valeurs brutes")   #tracé de la première courbe : valeurs brutes
plt.plot(moy2, label = "valeurs moyennes par mois")  #tracé de la seconde courbe : valeurs moyennes par mois
plt.legend()                               #afficher la légende 

plt.xlabel('Date')                        #légende de l'axe des abscisses 
plt.ylabel(param)                         #légénde de l'axe des ordonnées

### Calcul des valeurs maximales par jour

Calculons maintenant les valeurs maximales par jour

In [ ]:
ma = d_f.resample('D').max()  # 'D' pour day signifie par jour et max, maximum
ma

Idem que pour les calculs des moyennes, les dates des valeurs max sont toutes calées à une heure arbitraire, minuit d'après les premières dates affichées ci-dessous.

In [ ]:
print(ma.index[0])
print(ma.index[1])

Il est donc aussi utile de centrer les dates sur le milieu de la journée (12h00) afin de bien centrer les données sur les courbes.

In [ ]:
ma2 = ma.reset_index()  #remettre la date comme 'colonne classique' (la retirer de l'index)
ma2['date'] = ma2['date'].apply(lambda x: x + timedelta(hours=12))   #se mettre au milieu du jour : ajouter 12 heures
ma2.set_index('date', inplace=True)      #remettre la date en index -> colonne de référence
ma2

In [ ]:
plt.figure(figsize=(11,7))                  #taille de la fenêtre graphique
plt.plot(d_f, label = "valeurs brutes")   #tracé de la première courbe : valeurs brutes
plt.plot(ma2, label = "valeurs maximales par jour")  #tracé de la seconde courbe : valeurs maximales par jour
plt.legend()                               #afficher la légende 

plt.xlabel('Date')                        #légende de l'axe des abscisses 
plt.ylabel(param)                         #légénde de l'axe des ordonnées

Nous observons bien ici la courbe des valeurs maximales qui suit "l'enveloppe supérieure" des valeurs brutes.

### D'autres idées ? C'est à vous de jouer ! 

Vous pouvez jouer sur différents critères pour continuer de jouer avec les données, choisir : 

* une autre station ?
* un autre paramètre météo ?
* calculer les valeurs minimales ? par jour ? par mois ?
* ...etc

In [ ]:
#pour connaître les autres stations météo
df['number_sta'].value_counts()    #colonne de gauche : identifiant de la station, colonne de droite : nombre de lignes du tableau ayant cet identifiant